Для решения задачи рекомендации на основе MovieLens я выбрал библиотеку Pytorch и подход NCF. Мы будем тестироваться на рекомендации(подборке) 10 фильмов для каждого пользователя.Данные для обучения я предварительно загрузил на гугл диск для удобного доступа из colab. Далее, в классе UserItemRatingDataset я сделал представление наших данных в виде torch.tensor.

Я решил перевести Excplicit feedback в Implicit(т.е. оценки пользователями фильмов в бинарную метрику поставил оценку/не поставил), обычно Implicit feedbackа гораздо больше, поэтому было бы разумно сразу обучать модель так. 
В классе NCFData я создаю данные для обучения/тестирования. Важно отметить, что нам нужно вручную создать для пользователей отрицательные примеры(сейчас мы знаем только про положительные, т.е. оценки от пользователей). Для этого случайно будет добавлять к пользователем фильмы, которые он не оценил.
Для тестирования будем использовать подход Leave One Out, то есть тестировать на основе последней оценки фильма от пользователя.

К сожалению, при полной загрузке датасета MovieLens 25m вылетает оперативная память, так что я ограничусь примерно 500к оценок(если необходимо, при больших вычислительных мощностях можно засунуть и полный датасет).

Для оценки качества модели я выбрал метрики Hit(попадание релевантного элемента в рекомендуемые, без учета их ранжирования) и nDCG(normalized Discounted Cumulative Gain), где уже учитывается ранжирование(т.е. более релевантные элементы должны быть выше в списке рекомендаций). 

Далее я реализовал модель GMF(матричное разложение матрицы пользователей и фильмов). С помощью nn.Embeddings мы создаем два эмбеддинга пользователей и фильмов, перемножаем их поэлементно и пропускаем через полносвязный слой с активацией сигмоида.

В конце, обучаем нашу нейронную сеть с помощью train_pipeline и смотрим на средние значения метрик Hit и nDCG на каждой эпохе.
Уже после нескольких эпох мы достигаем неплохо качества Hit и nDCG(порядка 0.8 и 0.5 уже на 2 эпохе). Это достаточно хорошо, так как получается для 80% пользователей мы смогли порекомендовать такие фильмы, что они посмотрели и оценили хотя-бы один из них.

In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import random
from torch.utils.data import Dataset, DataLoader
import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TOP_K = 10

In [4]:
class UserItemRatingDataset(Dataset):
    """
    Делаем тензоры из наших данных
    """
    def __init__(self, user_list, item_list, rating_list):
        super(UserItemRatingDataset, self).__init__()
        self.user_tensor = torch.tensor(user_list, dtype=torch.long)
        self.item_tensor = torch.tensor(item_list, dtype=torch.long)
        self.target_tensor = torch.tensor(rating_list, dtype=torch.float)

    def __len__(self):
        return len(self.target_tensor)

    def __getitem__(self, idx):
        return self.user_tensor[idx], self.item_tensor[idx], self.target_tensor[idx]

In [5]:
class NCFData(object):
    """
    Собираем данные для обучения модели
    """

    def __init__(self, ratings, num_negatives, num_negatives_test, batch_size):
        self.ratings = ratings
        self.num_negatives = num_negatives
        self.num_negatives_test = num_negatives_test
        self.batch_size = batch_size

        self.preprocess_ratings = self._reindex(self.ratings)
        self.user_pool = set(self.ratings['user_id'].unique())
        self.item_pool = set(self.ratings['item_id'].unique())

        self.train_ratings, self.test_ratings = self._leave_one_out(self.preprocess_ratings)
        self.negatives = self._negative_sampling(self.preprocess_ratings)

    def _reindex(self, ratings):
        """
        Перенумеровываем данные, так как изначально у нас идут произвольные индексы
        """
        user_list = list(ratings['user_id'].drop_duplicates())
        self.user2id = {w: i for i, w in enumerate(user_list)}

        item_list = list(ratings['item_id'].drop_duplicates())
        self.item2id = {w: i for i, w in enumerate(item_list)}

        ratings['user_id'] = ratings['user_id'].apply(lambda x: self.user2id[x])
        ratings['item_id'] = ratings['item_id'].apply(lambda x: self.item2id[x])
        ratings['rating'] = ratings['rating'].apply(lambda x: float(x > 0))
        return ratings

    def _leave_one_out(self, ratings):
        """
        Хотим тестировать на последнем фильме, который пользователь оценил
        """
        ratings['rank_latest'] = ratings.groupby(['user_id'])['timestamp'].rank(method='first', ascending=False)
        test = ratings.loc[ratings['rank_latest'] == 1]
        train = ratings.loc[ratings['rank_latest'] > 1]
        test = test[test['user_id'].isin(train['user_id'].unique())]
        print(train['user_id'].nunique(), test['user_id'].nunique())
        assert train['user_id'].nunique() == test['user_id'].nunique(), 'Not Match Train User with Test User'
        return train[['user_id', 'item_id', 'rating']], test[['user_id', 'item_id', 'rating']]

    def _negative_sampling(self, ratings):
        """
        Для всех пользователей создаем какие-то негативные примеры(т.е. фильмы, которые он ещё не оценивал)
        """
        interact_status = (ratings.groupby('user_id')['item_id'].apply(set).reset_index().rename(
            columns={'item_id': 'interacted_items'}))
        interact_status['negative_items'] = (interact_status['interacted_items'].apply(lambda x: self.item_pool - x))
        interact_status['negative_samples'] = (
            interact_status['negative_items'].apply(lambda x: random.choices(tuple(x), k=self.num_negatives_test)))
        return interact_status[['user_id', 'negative_items', 'negative_samples']]

    def get_train_instance(self):
        """
        Собираем это все вместе и конструируем данные для обучения
        """
        users, items, ratings = [], [], []
        train_ratings = pd.merge(self.train_ratings, self.negatives[['user_id', 'negative_items']], on='user_id')
        train_ratings['negatives'] = train_ratings['negative_items'].apply(
            lambda x: random.choices(tuple(x), k=self.num_negatives))
        for row in train_ratings.itertuples():
            users.append(int(row.user_id))
            items.append(int(row.item_id))
            ratings.append(float(row.rating))
            for i in range(self.num_negatives):
                users.append(int(row.user_id))
                items.append(int(row.negatives[i]))
                ratings.append(float(0))  # negative samples get 0 rating

        dataset = UserItemRatingDataset(user_list=users, item_list=items, rating_list=ratings)
        return DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def get_test_instance(self):
        """
        Собираем данные примерно так же для тестирования
        """
        users, items, ratings = [], [], []
        test_ratings = pd.merge(self.test_ratings, self.negatives[['user_id', 'negative_samples']], on='user_id')
        for row in test_ratings.itertuples():
            users.append(int(row.user_id))
            items.append(int(row.item_id))
            ratings.append(float(row.rating))
            for i in getattr(row, 'negative_samples'):
                users.append(int(row.user_id))
                items.append(int(i))
                ratings.append(float(0))

        dataset = UserItemRatingDataset(user_list=users, item_list=items, rating_list=ratings)
        return DataLoader(dataset, batch_size=self.num_negatives_test + 1, shuffle=False, num_workers=2)

In [6]:
ml = pd.read_csv('drive/MyDrive/Colab Notebooks/ml-latest/ratings.csv').rename(columns={'userId': 'user_id', 'movieId': 'item_id'})

In [7]:
#Я изменил тип данных в каждой из колонок чтобы сэкономить оперативную память(но это не помогло)
#После этого просто обрезал датасет
ml['rating'] = (ml['rating'] * 2).astype('int8')
ml['user_id'] = ml['user_id'].astype('int32')
ml['item_id'] = ml['item_id'].astype('int32')
ml = ml.iloc[:27753444//50]

In [8]:
#получим количество фильмов и пользователей
num_users = ml['user_id'].nunique() + 1
num_items = ml['item_id'].nunique() + 1

In [9]:
data = NCFData(ml, num_negatives=4, num_negatives_test=100, batch_size=1024)

5498 5498


In [10]:
def hit(ng_item, pred_items):
    """
    Метрика попадание(без учета ранжирования)
    """
    if ng_item in pred_items:
        return 1
    return 0


def ndcg(ng_item, pred_items):
    """
    Метрика попадния с учетом ранжирования
    """
    if ng_item in pred_items:
        index = pred_items.index(ng_item)
        return np.reciprocal(np.log2(index + 2))
    return 0


@torch.no_grad()
def metrics(model, test_loader, top_k, device):
    """
    Соберем все метрики для каждого пользователя и усредним
    """
    _hr, _ndcg = [], []

    for user, item, label in test_loader:
        user = user.to(device)
        item = item.to(device)

        predictions = model(user, item)
        predictions = predictions.view(-1)
        _, indices = torch.topk(predictions, top_k)
        recommends = torch.take(item, indices).cpu().numpy().tolist()

        ng_item = item[0].item()  # leave one-out evaluation has only one item per user
        _hr.append(hit(ng_item, recommends))
        _ndcg.append(ndcg(ng_item, recommends))

    return np.mean(_hr), np.mean(_ndcg)

In [11]:
class GMF(nn.Module):
    """
    Строим архитектуру GMF как описывали в начале
    """
    def __init__(self, num_users, num_items, embedding_dim):
        super(GMF, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim

        self.embedding_user = nn.Embedding(num_embeddings=num_users, embedding_dim=embedding_dim)
        self.embedding_item = nn.Embedding(num_embeddings=num_items, embedding_dim=embedding_dim)
        self.affine_output = nn.Linear(in_features=embedding_dim, out_features=1)
        self.logistic = nn.Sigmoid()

        self.init_weight()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        """
        Для улучшения обучения и получения быстрее хороших значений метрик инициализируем веса с помощью Xavier Uniform
        """
        nn.init.xavier_uniform_(self.embedding_user.weight)
        nn.init.xavier_uniform_(self.embedding_item.weight)

In [12]:
def train_pipeline(model, optimizer, criterion, data, num_epochs):
    """
    Обучаем модель, с накоплением истории изменения метрик и удобным выводом с помощью tqdm
    """
    loss_history = []
    metrics_history = {'HR@10': [], 'NDCG@10': []}
    test_loader = data.get_test_instance()

    for epoch in range(1, num_epochs + 1):
        model.train() 
        train_loader = data.get_train_instance()

        for user, item, label in tqdm.tqdm(train_loader, desc=f'[Epoch #{epoch}]',total=len(train_loader)):
            user = user.to(DEVICE)
            item = item.to(DEVICE)
            label = label.to(DEVICE)

            optimizer.zero_grad()
            prediction = model(user, item)

            loss = criterion(prediction.view(-1), label.view(-1))
            loss.backward()
            optimizer.step()
            loss_history.append(loss.item())

        # Накапливаем метрики
        model.eval()
        hr_i, ndcg_i = metrics(model, test_loader, TOP_K, DEVICE)
        metrics_history['HR@10'].append(hr_i)
        metrics_history['NDCG@10'].append(ndcg_i)

        print(f"[Epoch #{epoch}] HR: {hr_i:.3f}\ndcg: {ndcg_i:.3f}")

    return loss_history, metrics_history

In [13]:
#Выбираем как критерий бинарную кросс энтропию что типично для задач бинарной классификации
# И стандартный оптимизатор Adam
model = GMF(num_users=num_users, num_items=num_items, embedding_dim=32)
model = model.to(DEVICE)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
loss_history, metrics_history = train_pipeline(model, optimizer, criterion, data, num_epochs=10)

[Epoch #1]: 100%|██████████| 2683/2683 [03:17<00:00, 13.61it/s]


[Epoch #1] HR: 0.784
dcg: 0.505


[Epoch #2]: 100%|██████████| 2683/2683 [03:19<00:00, 13.45it/s]


[Epoch #2] HR: 0.818
dcg: 0.539


KeyboardInterrupt: ignored